In [1]:
pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 8.3 MB/s eta 0:00:00


In [2]:
pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 24.5 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=bb0100042d8ef1356aff262e6180ba631f0209c0b7e92f734fe3b47211f5d05c
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=dff21ee69b5492bafe4524a826e83a1dd9e6dad536914af35d562df11eb8ee91
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [3]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=a86500e324b8b26443e7c3cfca4dedda1c9f881e18c752c2febe762c5a95ba26
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 14.1 MB/s eta 0:00:00


In [6]:
# Import Libraries
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import pandas as pd

from sumy.parsers.html import HtmlParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import requests

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize
import numpy as np

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.lsa import LsaSummarizer
from rouge_score import rouge_scorer

import faiss
import time
import numpy as np
import torch
import os
import pandas as pd
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Step 1: Import Data

In [7]:
headers = {
  'User-Agent': 'Dverma1@my.harrisburgu.edu'
}

response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'parse',
        'format': 'json',
        'page': 'Diabetes',
        'prop':'text',
        'redirects':''
    }).json()

raw_html = response['parse']['text']['*']
soup = BeautifulSoup(raw_html,'html.parser')
soup.find_all('p')
text = ''

paragraphs = []

for p in soup.find_all('p'):
    text = p.text
    text = re.sub('\\[[0-9]*\\]', '', text)
    text = re.sub('\\n', '', text)
    paragraphs.append(text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
summary = paragraphs[0:2]
main_text = paragraphs[2:]
summary = ' '.join(summary)
main_text = ' '.join(main_text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Step 2: Create A Search Engine

In [10]:
data = pd.DataFrame(paragraphs, columns = ['sentence'])
data = data.drop(0)
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sentence
1,"Diabetes mellitus, often known simply as diabe..."
2,The major types of diabetes are type 1 and typ...
3,"As of 2021, an estimated 537 million people ha..."
4,The classic symptoms of untreated diabetes are...
5,Diabetic ketoacidosis is a medical emergency t...


In [11]:
## Load a pre-trained model
model = SentenceTransformer('msmarco-MiniLM-L-12-v3')
article = data["sentence"].tolist()
article_nlp = model.encode(article)

# Create an index using FAISS
index = faiss.IndexFlatL2(article_nlp.shape[1])
index.add(article_nlp)
faiss.write_index(index, 'index_article')
index = faiss.read_index('index_article')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# define a search
def search(query):

    t=time.time()
    query_vector = model.encode([query])
    k = 5
    top_k = index.search(query_vector, k)
    print('totaltime: {}'.format(time.time()-t))
    return [article[_id] for _id in top_k[1].tolist()[0]]

# use the search
results = search("symptoms")

totaltime: 0.044490814208984375


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
results

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['The symptoms may relate to fluid loss and polyuria, but the course may also be insidious. Diabetic animals are more prone to infections. The long-term complications recognized in humans are much rarer in animals. The principles of treatment (weight loss, oral antidiabetics, subcutaneous insulin) and management of emergencies (e.g. ketoacidosis) are similar to those in humans.',
 "Diabetes mellitus, often known simply as diabetes, is a group of common endocrine diseases characterized by sustained high blood sugar levels. Diabetes is due to either the pancreas not producing enough insulin, or the cells of the body becoming unresponsive to the hormone's effects. Classic symptoms include thirst, polyuria, weight loss, and blurred vision. If left untreated, the disease can lead to various health complications, including disorders of the cardiovascular system, eye, kidney, and nerves. Diabetes accounts for approximately 4.2 million deaths every year, with an estimated 1.5 million caused by

In [15]:
#Search for several items (at least three words or phrases).
import time

def search(queries):
    results = {}
    for query in queries:
        t = time.time()
        query_vector = model.encode([query])
        k = 5
        top_k = index.search(query_vector, k)
        print('Total time for "{}": {:.2f} seconds'.format(query, time.time() - t))
        results[query] = [article[_id] for _id in top_k[1].tolist()[0]]
    return results

# Use the search function for several items
queries = ["medical emergency", "Complications of diabetes", "World Health Organization"]
results = search(queries)

# Display the results
for query, res in results.items():
    print(f"\nResults for '{query}':")
    for item in res:
        print(item)


Total time for "medical emergency": 0.05 seconds
Total time for "Complications of diabetes": 0.04 seconds
Total time for "World Health Organization": 0.04 seconds

Results for 'medical emergency':
In 2010, diabetes-related emergency room (ER) visit rates in the United States were higher among people from the lowest income communities (526 per 10,000 population) than from the highest income communities (236 per 10,000 population). Approximately 9.4% of diabetes-related ER visits were for the uninsured.
Diabetic ketoacidosis is a medical emergency that occurs most commonly in type 1, but may also occur in type 2 if it has been longstanding or if the individual has significant β-cell dysfunction. Excessive production of ketone bodies leads to signs and symptoms including nausea, vomiting, abdominal pain, the smell of acetone in the breath, deep breathing known as Kussmaul breathing, and in severe cases decreased level of consciousness. Hyperosmolar hyperglycemic state is another emergency

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Step 3: Create Text Summary

In [17]:
import nltk
nltk.download('punkt')
main_text_sent = tokenize.sent_tokenize(main_text)
num_summary_sentence = 4

##Text Rank Summary
textrank_summary = []

parser = PlaintextParser.from_string(main_text, Tokenizer("english"))
summarizer = TextRankSummarizer()
for sentence in summarizer(parser.document, num_summary_sentence):
    textrank_summary.append(str(sentence))


##LSA Summary
lsa_summary = []

LANGUAGE = "english"
stemmer = Stemmer(LANGUAGE)

parser = PlaintextParser.from_string(main_text, Tokenizer(LANGUAGE))
summarizer = LsaSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, num_summary_sentence):
    lsa_summary.append(str(sentence))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [32]:
#LDA Summary
import pandas as pd
nltk.download('stopwords')
#tokenize, remove stopwords, non-alphabetic words, lowercase
def preprocess(textstring):
   stops =  set(stopwords.words('english'))
   tokens = word_tokenize(textstring)
   return [token.lower() for token in tokens if token.isalpha()
          and token not in stops]

processed_sentences = [preprocess(sent) for sent in main_text_sent]

dictionary = Dictionary(processed_sentences)
corpus = [dictionary.doc2bow(sent) for sent in processed_sentences]

LDAmodel = LdaModel(corpus = corpus,
                id2word = dictionary,
                iterations = 400,
                num_topics = 5,
                random_state = 100,
                update_every = 1,
                chunksize = 100,
                passes = 10,
                alpha = 'auto',
                per_word_topics = True)

probs = [LDAmodel.get_document_topics(sentence) for sentence in corpus]

save_probs = []
i = 0
for document in probs:
  for (topic, prob) in document:
    if topic == 1:
      save_probs.append((main_text_sent[i], prob))
  i = i + 1

DF = pd.DataFrame(save_probs, columns = ["sentence", "prob"])
lda_summary = "".join(DF.sort_values(by = ["prob"], ascending = False)[0:3].sentence)

lda_summary

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'The following is a list of disorders that may increase the risk of diabetes: Insulin is the principal hormone that regulates the uptake of glucose from the blood into most cells of the body, especially liver, adipose tissue and muscle, except smooth muscle, in which insulin acts via the IGF-1.A 2016 systematic review found potential harm to treating to targets lower than 140 mmHg, and a subsequent systematic review in 2019 found no evidence of additional benefit from blood pressure lowering to between 130 – 140mmHg, although there was an increased risk of adverse events.The body obtains glucose from three main sources: the intestinal absorption of food; the breakdown of glycogen (glycogenolysis), the storage form of glucose found in the liver; and gluconeogenesis, the generation of glucose from non-carbohydrate substrates in the body.'

In [46]:
# Show top words for each topic
topics = LDAmodel.show_topics(num_topics=5, num_words=10, formatted=False)
for topic_num, topic in topics:
    print(f"Topic {topic_num}:")
    print(", ".join([word for word, prob in topic]))

Topic 0:
meaning, symptoms, the, disease, also, may, word, treatment, insulin, comes
Topic 1:
diabetes, glucose, type, risk, the, blood, higher, people, body, lower
Topic 2:
diabetes, type, mellitus, also, likely, affected, time, breeds, in, due
Topic 3:
diabetes, disease, sugar, insulin, type, cardiovascular, blood, complications, people, evidence
Topic 4:
diabetes, the, first, cases, insulin, less, glucose, types, early, management


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic 0: General information about diabetes, including definitions, symptoms, and treatments.

Topic 1: The relationship between diabetes, blood glucose levels, and associated risk factors.

Topic 2: Different types of diabetes and the demographics affected by it.
Topic 3: Complications of diabetes, particularly cardiovascular issues.

Topic 4: The history of diabetes and its management.

In [33]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt

vis = pyLDAvis.gensim_models.prepare(LDAmodel, corpus, dictionary, n_jobs = 1)
pyLDAvis.save_html(vis, 'LDA_Visualization_diabetes.html') ##saves the file

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
def print_rouge_score(rouge_score):
    for k,v in rouge_score.items():
        print (k, 'Precision:', "{:.2f}".format(v.precision), 'Recall:', "{:.2f}".format(v.recall), 'fmeasure:', "{:.2f}".format(v.fmeasure))

textrank_sum = "".join(str(textrank_summary))

scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
scores = scorer.score(summary, textrank_sum)
print_rouge_score(scores)

rouge1 Precision: 0.19 Recall: 0.37 fmeasure: 0.25


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
lsa_sum = "".join(str(lsa_summary))

scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
scores = scorer.score(summary, lsa_sum)
print_rouge_score(scores)

rouge1 Precision: 0.20 Recall: 0.34 fmeasure: 0.26


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
scores = scorer.score(summary, lda_summary)
print_rouge_score(scores)

rouge1 Precision: 0.20 Recall: 0.27 fmeasure: 0.23


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Step 4: Intepretation

- #### **What did you learn about the chosen webpage from the above analysis? How well did the search engine work? What could make it better? How well does the text summary compare to your understanding of the text? (Write at least 5 sentences)**

## Answer

### 1. **Webpage:** From the above analysis, I learned that the chosen webpage contains detailed information on diabetes, covering various aspects such as general definitions, symptoms, treatments, blood glucose levels, risk factors, types of diabetes, complications (particularly cardiovascular), and historical management.

### 2. **Search Engine:** The search engine effectively retrieved relevant information for the queried topics: "medical emergency," "Complications of diabetes," and "World Health Organization." Each query returned paragraphs closely related to the respective topics, discussing emergency situations related to diabetes, various complications, and global diabetes statistics and management. The search queries were processed quickly, with each query taking less than 0.05 seconds, indicating efficient retrieval from the indexed data.

### 3. **Improving Search Engine:** Implementing techniques to expand queries with synonyms or related terms could improve the search engine's ability to capture broader aspects of the topics. Also, enhancing the model's ability to understand the context of queries, especially in medical or technical domains like diabetes, could refine result relevance.

### 4. **Text Summary:** The summaries generated using LSA and LDA methods align well with the key themes identified in the text analysis, as reflected in the slightly higher ROUGE-1 precision scores. These methods successfully distilled the essential information, providing a coherent and relevant summary that matches the detailed content on the webpage.

#### ROUGE-1 Precision Scores:

#### TextRank Summary: Precision 0.19

#### LSA Summary: Precision 0.20

#### LDA Summary: Precision 0.20

## Step 5: Application

#### **Describe some real-world application that could use the techniques from this section. Define a problem or question, what data would be needed, how the data would be analyzed, and what insights/solutions could be drawn from the analysis.**

## Answer

### One of such real-world application would be involving **Academic Research Literature Review.**

### **Problem:** Researchers need to conduct comprehensive literature reviews across multiple academic journals and publications to identify gaps in knowledge, emerging trends, and relevant studies for their research.

### **Data Needed:** Academic papers, journal articles, conference proceedings, and research abstracts.

### **Data Analysis Techniques:** Text mining for keyword extraction and topic modeling to cluster research articles by themes; citation analysis to identify influential papers and authors; and summarization to distill key findings from extensive literature.

### **Insights/Solutions:** Insights can help researchers stay updated on the latest advancements, identify collaborators, and formulate research hypotheses based on gaps in current knowledge.